In [1]:
#####Bayesian multinomial rank probit model#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import seaborn as sns
from scipy import sparse
from scipy.stats import norm
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize

#np.random.seed(98537)

In [2]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no, pattern):
    z_id = np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1)
    if pattern==1:
        Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
        return z_id, Z
    return z_id

In [3]:
##任意の相関行列(分散共分散行列)を作成する関数####
#任意の相関行列を作る関数
def CorM(col, lower, upper, eigen_lower, eigen_upper):
    #相関行列の初期値を定義する
    cov_vec = np.random.uniform(lower, upper, col*col)   #相関係数の乱数ベクトルを作成
    rho = np.tril(cov_vec.reshape(col, col), k=-1)   #乱数ベクトルを下三角行列化
    Sigma = (rho + rho.T) + np.diag(np.repeat(1, col))   #対角成分を1にする
    
    #相関行列を正定値行列に変更
    #固有値分解を実行
    eigen = np.linalg.eigh(Sigma)
    eigen_val = eigen[0] 
    eigen_vec = eigen[1]
    
    #固有値が負の数値を正にする
    for i in range(col):
        if eigen_val[i] < 0:
            eigen_val[i] = np.random.uniform(eigen_lower, eigen_upper, 1)
            
    #新しい相関行列の定義と対角成分を1にする
    Sigma = np.dot(np.dot(eigen_vec, np.diag(eigen_val)), eigen_vec.T)
    normalization_factor = np.dot(np.power(np.diag(Sigma), 0.5)[:, np.newaxis], np.power(np.diag(Sigma), 0.5)[np.newaxis, :])
    Cor = Sigma / normalization_factor
    return Cor

#相関行列から分散共分散行列に変換する関数
def covmatrix(Cor, sigma_lower, sigma_upper):
    sigma = (sigma_upper - sigma_lower) * rand(np.diag(Cor).shape[0]) + sigma_lower
    sigma_factor = np.dot(sigma[:, np.newaxis], sigma[np.newaxis, :])
    Cov = Cor * sigma_factor
    return Cov

#分散共分散行列から相関行列に変換する関数
def cov2cor(Cov):
    D = np.diag(np.power(np.diag(Cov), -1/2))
    corr = np.dot(np.dot(D, Cov), D)
    return corr

In [4]:
##データの設定
r = 3   #3位まで選択
n = 20000   #ユーザー数
member = 10   #選択可能数
N = member*n   #総レコード数

In [5]:
##idとインデックスの設定
#idの設定
user_id1 = np.repeat(np.arange(n), np.repeat(member, n))
user_id2 = np.repeat(np.arange(n), np.repeat(member-1, n))
member_id = np.tile(np.arange(member), n)

#インデックスの設定
user_list1 = [i for i in range(n)]
user_list2 = [i for i in range(n)]
member_list = [j for j in range(member)]
for i in range(n):
    user_list1[i] = np.array(np.where(user_id1==i)[0], dtype="int")
    user_list2[i] = np.array(np.where(user_id2==i)[0], dtype="int")
for j in range(member):
    member_list[j] = np.array(np.where(member_id==j)[0], dtype="int")
index_standard = member_list[member-1]
index_target = np.delete(np.arange(N), index_standard)
no_target = np.repeat(np.arange(n), member-1)

In [6]:
##説明変数の生成
#切片の設定
Intercept = np.vstack((np.diag(np.repeat(1, member-1)), np.repeat(0, member-1)))[member_id, ]

#登録からの経過年数
regist = np.abs(np.random.normal(0, 1.0, n))
REGIST = Intercept * regist[user_id1, np.newaxis]

#プレイ時間の対数
play = np.abs(np.random.normal(0, 1.0, n))
PLAY = Intercept * play[user_id1, np.newaxis]

#衣装タイプの生成
types = 9   #衣装数
CLOTH = np.zeros((N, types), dtype="int")
Prob = np.random.dirichlet(np.repeat(2.0, types), 1).reshape(-1)
for j in range(member):
    CLOTH[member_list[j], ] = np.random.multinomial(1, Prob, n)
CLOTH = CLOTH[:, np.delete(np.arange(types), np.argmin(np.sum(CLOTH, axis=0)))]
CLOTH[index_target, ] = CLOTH[index_target, ] - CLOTH[index_standard[no_target], ]   #相対変数に変換
CLOTH[index_standard, ] = 0

#メンバーのshare of voiceを生成
alpha = np.abs(np.random.normal(0, 0.5, member))
AWARE = np.random.dirichlet(alpha, n).reshape(-1)
AWARE[index_target] = AWARE[index_target] - AWARE[index_standard[no_target]]   #相対変数に変換
AWARE[index_standard] = 0

#メンバーの接触確率を生成
alpha = np.abs(np.random.normal(0, 0.5, member))
CONTACT = np.random.dirichlet(alpha, n).reshape(-1)
CONTACT[index_target] = CONTACT[index_target] - CONTACT[index_standard[no_target]]   #相対変数に変換
CONTACT[index_standard] = 0

#データの結合
x1 = np.hstack((Intercept, REGIST, PLAY))
x2 = np.hstack((CLOTH, AWARE[:, np.newaxis], CONTACT[:, np.newaxis]))
x = np.array(np.hstack((x1, x2)), dtype="float32")
k1 = x1.shape[1]
k2 = x2.shape[1]
k = x.shape[1]
del Intercept, REGIST, PLAY, CLOTH, AWARE, CONTACT

In [7]:
##応答変数の生成
rp = 0
while True:
    rp = rp + 1
    print(rp)
    
    #分散共分散行列のパラメータを生成
    Cov = CorM(member-1, -0.6, 0.9, 0.1, 1.0)
    Covt = Cov.copy()

    #回帰パラメータを生成
    beta01 = np.random.multivariate_normal(np.repeat(0.75, member-1), np.diag(np.repeat(1.5, member-1)), 1).reshape(-1)
    beta02 = np.random.multivariate_normal(np.repeat(0.0, member-1), np.diag(np.repeat(0.25, member-1)), 1).reshape(-1)
    beta03 = np.random.multivariate_normal(np.repeat(0.0, member-1), np.diag(np.repeat(0.25, member-1)), 1).reshape(-1)
    beta11 = np.random.multivariate_normal(np.repeat(0.0, types-1), np.diag(np.repeat(1.0, types-1)), 1).reshape(-1)
    beta12 = np.array([0.75])
    beta13 = np.array([1.0])
    beta = np.concatenate((beta01, beta02, beta03, beta11, beta12, beta13))
    betat = beta.copy()

    #潜在効用を定義
    mu = np.dot(x, beta).reshape(n, member)
    U = np.hstack((mu[:, :member-1] + np.random.multivariate_normal(np.repeat(0, member-1), Cov, n), np.repeat(0, n)[:, np.newaxis]))
    mut = mu.copy(); UT = U.copy()

    #効用最大化原理に基づき相対順位を決定
    Rank = np.array(np.argsort(U, axis=1)[:, ::-1], dtype="int")
    Rank_target = np.array(Rank[:, :r], dtype="int")

    #break条件
    freq = np.array([np.sum(Rank_target==i) for i in range(member)])
    if (np.min(freq) > n/(k*r)) & (np.min(freq) < n/k):
        break
        
#生成したデータを確認
print(freq)
print(np.round(freq / np.sum(freq), 3))

1
[ 4987  7950   333 15151  8357  6054  8738   247  6261  1922]
[0.083 0.132 0.006 0.253 0.139 0.101 0.146 0.004 0.104 0.032]


In [8]:
#ランクのインデックス
rank_index1 = [i for i in range(member)]
rank_index2 = [i for i in range(member)]
for i in range(member):
    rank_index1[i] = np.array(np.where(np.sum(np.array(Rank_target!=i, dtype="int"), axis=1)==r)[0], dtype="int")
    temp_index = [j for j in range(r)]
    for j in range(r):
        temp_index[j] = np.array(np.where(Rank_target[:, j]==i)[0], dtype="int")
    rank_index2[i] = temp_index

In [9]:
####マルコフ連鎖モンテカルロ法でランクプロビットモデルを推定####
##パラメータ推定のための関数を定義
#切断正規分布の乱数を発生させる関数
def rtnorm(mu, sigma, a, b, n):
    FA = norm.cdf(a, mu, sigma)
    FB = norm.cdf(b, mu, sigma)
    return norm.ppf(np.random.uniform(0, 1, n)*(FB-FA)+FA, mu, sigma)

#多変量正規分布の条件付き期待値と分散を計算する関数
def cdMVN(mu, Cov, department, U):
    #分散共分散行列のブロック行列を定義
    department = np.array([department])
    index = np.delete(np.arange(Cov.shape[0]), department)
    Cov11 = Cov[department, ][:, department]
    Cov12 = Cov[department, ][:, index]
    Cov21 = Cov[:, department][index, ]
    Cov22 = Cov[index, ][:, index]

    #条件付き分散と条件付き平均を計算
    inv_Cov22 = np.linalg.inv(Cov22)
    CDinv = np.dot(Cov12, inv_Cov22)
    CDmu = mu[:, department] + np.dot(CDinv, (U[:, index] - mu[:, index]).T).T   #条件付き平均
    CDvar = Cov11 - np.dot(np.dot(Cov12, inv_Cov22), Cov21)   #条件付き分散
    return CDmu, CDvar

#多変量正規分布の密度関数
def mvdnorm(u, mu, Cov, k):
    er = U - mu 
    inv_Cov = np.linalg.inv(Cov)
    det_Cov = np.linalg.det(Cov)
    Lho = 1 / (np.power(np.sqrt(2*np.pi), k)*det_Cov) * np.exp(-1/2 * np.sum(np.dot(er, inv_Cov) * er, axis=1))
    return Lho 

In [10]:
##アルゴリズムの設定
R = 5000
keep = 4
burnin = int(500/keep)
iter = 0
disp = 10
e = 0.05
L = 3
m_vec = np.repeat(1, member-1)

In [11]:
##データの設定
#除外変数の定義
exclusion = np.zeros((member-1, member-2), dtype="int")
for j in range(member-1):
    exclusion[j, ] = np.delete(np.arange(member-1), j)

#基準変数のフラグを建てる
flag1 = np.array(Rank_target!=member-1, dtype="int")
flag2 = 1 - flag1

#データの定数
Data = np.zeros((member-1, k, n), dtype="float32")
for i in range(n):
    Data[:, :, i] = x[user_list1[i][np.arange(member-1)], ]

In [12]:
##事前分布の設定
#回帰係数の事前分布
Deltabar = np.repeat(0, k)
ADelta = 0.01 * np.diag(np.repeat(1.0, k))

#分散共分散行列の事前分布
nu = 1.0
V = 0.1 * np.diag(np.repeat(1.0, member-1))

In [13]:
##パラメータの真値
#モデルパラメータの真値
Cov = Covt.copy()
inv_Cov = np.linalg.inv(Cov)
beta = betat.copy()

#効用の平均構造の真値
mu = mut.copy()
U = UT.copy()
rank_u = np.zeros((n, r))

In [14]:
##パラメータの初期値
#モデルパラメータの初期値
Cov = np.diag(np.repeat(1.0, member-1))
inv_Cov = np.linalg.inv(Cov)
beta = np.repeat(0.0, k)

#効用の平均構造の初期値
mu = np.dot(x, beta).reshape(n, member)
U = np.hstack((mu[:, :member-1] + np.random.multivariate_normal(np.repeat(0, member-1), np.diag(np.repeat(0.2, member-1)), n), 
               np.repeat(0, n)[:, np.newaxis]))
rank_u = np.zeros((n, r))

In [15]:
##パラメータの格納用配列
BETA = np.zeros((int(R/keep), k))
COV = np.zeros((member-1, member-1, int(R/keep)))

In [16]:
####ギブスサンプリングでパラメータをサンプリング####
for rp in range(R):
    
    ##順位選択結果と整合的な潜在効用を発生させる
    #条件付き期待値と条件付き分散を計算
    util_mu = np.zeros((n, member-1))
    S = np.repeat(0.0, member-1)
    for i in range(member-1):
        MVR = cdMVN(mu, Cov, i, U)   #多変量正規分布の条件付き分布  
        util_mu[:, i] = MVR[0].reshape(-1)   #条件付き期待値を抽出
        S[i] = np.sqrt(MVR[1].reshape(-1))   #条件付き分散を抽出

        #潜在効用の切断領域の定義
        Util = np.hstack((U[:, exclusion[i, ]], np.zeros(n)[:, np.newaxis]))
        target = (np.argsort(Util, axis=1)[:, ::-1])[:, :r]
        for j in range(r):
            index = rank_index2[i][j]
            rank_u[:, j] = np.dot(Util * np.eye(member-1, dtype="int")[target[:, j]], m_vec)
            rank_u[:, j] = flag1[:, j]*rank_u[:, j] + flag2[:, j]*0

            #切断正規分布から潜在効用を生成
            if j==0:
                U[index, i] = rtnorm(util_mu[index, i], S[i], rank_u[index, j], 100, index.shape[0])   #1位の潜在効用
            else:
                U[index, i] = rtnorm(util_mu[index, i], S[i], rank_u[index, j], rank_u[index, j-1], index.shape[0])   #2位～3位の潜在効用
            index = rank_index1[i]
            U[index, i] = rtnorm(util_mu[index, i], S[i], -100, rank_u[index, r-1], index.shape[0])   #それ以降の潜在効用


    ##SURモデルにより回帰ベクトルをサンプリング
    #入力変数と応答変数を定義
    Chol = np.linalg.cholesky(inv_Cov)   #分散共分散行列の逆行列のコレツキー分解
    X = np.zeros((user_id2.shape[0], k))
    for i in range(n):
        X[user_list2[i], ] = np.dot(Chol, Data[:, :, i])
    u = (np.dot(Chol, U[:, :member-1].T).T).reshape(-1)   #潜在効用をベクトルに変換

    #多変量正規分布から事後分布をサンプリング
    inv_XXV = np.linalg.inv(np.dot(X.T, X) + ADelta)
    Xy = np.dot(X.T, u)
    beta_mu = np.dot(inv_XXV, Xy + np.dot(ADelta, Deltabar))
    beta = np.random.multivariate_normal(beta_mu, inv_XXV, 1).reshape(-1)


    ##逆ウィシャート分布から分散共分散行列をサンプリング
    #モデル誤差を定義
    mu = np.dot(x, beta).reshape(n, member)
    er = (U - mu)[:, :member-1]

    #逆ウィシャート分布のパラメータ
    IW_R = np.dot(er.T, er) + V
    Sn = n + nu 

    #事後分布をサンプリング
    Cov = scipy.stats.invwishart.rvs(Sn, IW_R, 1)
    Cov = cov2cor(Cov)
    inv_Cov = np.linalg.inv(Cov)
    
    
    ##サンプリング結果の保存と表示
    if rp%keep==0:
        #サンプリング結果の格納
        mkeep = int(rp/keep)
        BETA[mkeep, ] = beta
        COV[:, :, mkeep] = Cov
        
    #サンプリング結果の表示
    if rp%disp==0:
        print(rp)
        print(np.round(np.vstack((beta, betat)), 2))

0
[[-0.27  0.4  -0.62  0.6  -0.25 -0.32  0.09 -0.6  -0.38 -0.06 -0.27  0.
   0.08  0.12 -0.03 -0.32 -0.01  0.23  0.   -0.4   0.02 -0.23  0.12  0.11
   0.17  0.   -0.04  0.02 -0.14 -0.06  0.13 -0.18  0.01 -0.05  0.02  0.13
   0.22]
 [ 0.73  2.16 -1.94  2.07  0.35  0.45  1.12 -1.9   0.23 -0.4  -0.85  0.24
  -0.09  0.03 -0.29 -0.87  0.46  0.34 -0.07 -1.08  0.61 -0.33  0.33  0.3
   0.36 -0.48 -0.14  0.21 -0.73 -0.22  0.66 -0.94  0.14 -0.19  0.16  0.75
   1.  ]]
10
[[-0.    1.33 -1.71  1.32 -0.24 -0.35  0.39 -1.52 -0.47 -0.23 -0.65  0.11
   0.06  0.18 -0.14 -0.67  0.1   0.49 -0.03 -0.98  0.26 -0.3   0.3   0.35
   0.39 -0.05 -0.12  0.15 -0.42 -0.14  0.49 -0.52  0.11 -0.12  0.12  0.39
   0.68]
 [ 0.73  2.16 -1.94  2.07  0.35  0.45  1.12 -1.9   0.23 -0.4  -0.85  0.24
  -0.09  0.03 -0.29 -0.87  0.46  0.34 -0.07 -1.08  0.61 -0.33  0.33  0.3
   0.36 -0.48 -0.14  0.21 -0.73 -0.22  0.66 -0.94  0.14 -0.19  0.16  0.75
   1.  ]]
20
[[ 0.2   1.59 -2.02  1.53 -0.09 -0.16  0.54 -1.73 -0.41 -0.27 -0.76  0

180
[[ 0.67  2.17 -2.47  2.07  0.33  0.43  1.11 -1.91  0.2  -0.37 -0.91  0.32
  -0.09  0.02 -0.33 -0.87  0.43  0.31 -0.09 -1.1   0.74 -0.37  0.29  0.28
   0.35 -0.3  -0.14  0.19 -0.75 -0.24  0.6  -0.98  0.12 -0.23  0.14  0.78
   0.95]
 [ 0.73  2.16 -1.94  2.07  0.35  0.45  1.12 -1.9   0.23 -0.4  -0.85  0.24
  -0.09  0.03 -0.29 -0.87  0.46  0.34 -0.07 -1.08  0.61 -0.33  0.33  0.3
   0.36 -0.48 -0.14  0.21 -0.73 -0.22  0.66 -0.94  0.14 -0.19  0.16  0.75
   1.  ]]
190
[[ 0.68  2.14 -2.46  2.03  0.32  0.38  1.02 -1.94  0.11 -0.4  -0.89  0.33
  -0.1   0.01 -0.31 -0.87  0.44  0.34 -0.07 -1.08  0.74 -0.3   0.32  0.32
   0.4  -0.22 -0.11  0.18 -0.75 -0.23  0.63 -0.97  0.12 -0.2   0.13  0.75
   0.96]
 [ 0.73  2.16 -1.94  2.07  0.35  0.45  1.12 -1.9   0.23 -0.4  -0.85  0.24
  -0.09  0.03 -0.29 -0.87  0.46  0.34 -0.07 -1.08  0.61 -0.33  0.33  0.3
   0.36 -0.48 -0.14  0.21 -0.73 -0.22  0.66 -0.94  0.14 -0.19  0.16  0.75
   1.  ]]
200
[[ 0.63  2.12 -2.51  2.07  0.3   0.36  1.09 -1.95  0.15 -0.37 -0

360
[[ 0.69  2.12 -2.32  2.07  0.36  0.44  1.11 -1.89  0.16 -0.42 -0.88  0.33
  -0.1   0.01 -0.33 -0.87  0.51  0.35 -0.04 -1.01  0.72 -0.33  0.32  0.31
   0.36 -0.46 -0.1   0.17 -0.74 -0.25  0.59 -0.98  0.09 -0.23  0.13  0.76
   0.95]
 [ 0.73  2.16 -1.94  2.07  0.35  0.45  1.12 -1.9   0.23 -0.4  -0.85  0.24
  -0.09  0.03 -0.29 -0.87  0.46  0.34 -0.07 -1.08  0.61 -0.33  0.33  0.3
   0.36 -0.48 -0.14  0.21 -0.73 -0.22  0.66 -0.94  0.14 -0.19  0.16  0.75
   1.  ]]
370
[[ 0.67  2.16 -2.3   2.03  0.35  0.44  1.1  -1.8   0.22 -0.39 -0.89  0.34
  -0.09  0.01 -0.31 -0.86  0.47  0.32 -0.06 -1.07  0.7  -0.29  0.32  0.31
   0.38 -0.55 -0.1   0.18 -0.74 -0.23  0.62 -0.97  0.12 -0.24  0.15  0.78
   0.98]
 [ 0.73  2.16 -1.94  2.07  0.35  0.45  1.12 -1.9   0.23 -0.4  -0.85  0.24
  -0.09  0.03 -0.29 -0.87  0.46  0.34 -0.07 -1.08  0.61 -0.33  0.33  0.3
   0.36 -0.48 -0.14  0.21 -0.73 -0.22  0.66 -0.94  0.14 -0.19  0.16  0.75
   1.  ]]
380
[[ 0.7   2.16 -2.26  2.09  0.37  0.45  1.12 -1.75  0.24 -0.41 -0

540
[[ 0.71  2.17 -2.5   2.09  0.36  0.46  1.1  -1.85  0.2  -0.4  -0.85  0.38
  -0.09  0.03 -0.3  -0.87  0.45  0.35 -0.06 -1.08  0.78 -0.34  0.3   0.28
   0.36 -0.47 -0.13  0.19 -0.75 -0.25  0.62 -1.    0.11 -0.24  0.14  0.76
   0.98]
 [ 0.73  2.16 -1.94  2.07  0.35  0.45  1.12 -1.9   0.23 -0.4  -0.85  0.24
  -0.09  0.03 -0.29 -0.87  0.46  0.34 -0.07 -1.08  0.61 -0.33  0.33  0.3
   0.36 -0.48 -0.14  0.21 -0.73 -0.22  0.66 -0.94  0.14 -0.19  0.16  0.75
   1.  ]]
550
[[ 0.68  2.17 -2.37  2.1   0.42  0.43  1.12 -1.84  0.22 -0.38 -0.87  0.33
  -0.1  -0.   -0.31 -0.86  0.42  0.31 -0.04 -1.07  0.75 -0.36  0.28  0.31
   0.36 -0.47 -0.12  0.2  -0.75 -0.23  0.63 -0.98  0.11 -0.27  0.15  0.78
   1.  ]
 [ 0.73  2.16 -1.94  2.07  0.35  0.45  1.12 -1.9   0.23 -0.4  -0.85  0.24
  -0.09  0.03 -0.29 -0.87  0.46  0.34 -0.07 -1.08  0.61 -0.33  0.33  0.3
   0.36 -0.48 -0.14  0.21 -0.73 -0.22  0.66 -0.94  0.14 -0.19  0.16  0.75
   1.  ]]
560
[[ 0.71  2.15 -2.41  2.1   0.38  0.46  1.12 -1.82  0.22 -0.4  -0

720
[[ 0.71  2.17 -2.12  2.09  0.41  0.46  1.16 -1.98  0.2  -0.38 -0.89  0.31
  -0.1   0.01 -0.32 -0.86  0.44  0.35 -0.08 -1.08  0.67 -0.35  0.28  0.29
   0.34 -0.32 -0.15  0.22 -0.72 -0.22  0.64 -0.94  0.13 -0.23  0.16  0.74
   0.99]
 [ 0.73  2.16 -1.94  2.07  0.35  0.45  1.12 -1.9   0.23 -0.4  -0.85  0.24
  -0.09  0.03 -0.29 -0.87  0.46  0.34 -0.07 -1.08  0.61 -0.33  0.33  0.3
   0.36 -0.48 -0.14  0.21 -0.73 -0.22  0.66 -0.94  0.14 -0.19  0.16  0.75
   1.  ]]
730
[[ 0.73  2.18 -2.02  2.09  0.38  0.49  1.15 -1.93  0.22 -0.4  -0.87  0.29
  -0.11  0.06 -0.33 -0.87  0.44  0.33 -0.08 -1.08  0.6  -0.34  0.26  0.27
   0.34 -0.36 -0.12  0.21 -0.74 -0.22  0.63 -0.95  0.13 -0.24  0.16  0.78
   0.98]
 [ 0.73  2.16 -1.94  2.07  0.35  0.45  1.12 -1.9   0.23 -0.4  -0.85  0.24
  -0.09  0.03 -0.29 -0.87  0.46  0.34 -0.07 -1.08  0.61 -0.33  0.33  0.3
   0.36 -0.48 -0.14  0.21 -0.73 -0.22  0.66 -0.94  0.14 -0.19  0.16  0.75
   1.  ]]
740
[[ 0.73  2.15 -2.06  2.11  0.4   0.47  1.15 -1.91  0.2  -0.42 -0

...

4140
[[ 0.71  2.14 -2.15  2.07  0.38  0.44  1.14 -1.77  0.19 -0.41 -0.87  0.29
  -0.09  0.01 -0.33 -0.89  0.42  0.34 -0.04 -1.05  0.68 -0.34  0.3   0.32
   0.38 -0.41 -0.1   0.2  -0.76 -0.25  0.61 -0.98  0.12 -0.25  0.14  0.76
   1.  ]
 [ 0.73  2.16 -1.94  2.07  0.35  0.45  1.12 -1.9   0.23 -0.4  -0.85  0.24
  -0.09  0.03 -0.29 -0.87  0.46  0.34 -0.07 -1.08  0.61 -0.33  0.33  0.3
   0.36 -0.48 -0.14  0.21 -0.73 -0.22  0.66 -0.94  0.14 -0.19  0.16  0.75
   1.  ]]
4150
[[ 0.71  2.16 -2.16  2.11  0.38  0.46  1.12 -1.86  0.18 -0.4  -0.86  0.32
  -0.09  0.03 -0.33 -0.86  0.48  0.34 -0.06 -1.07  0.63 -0.39  0.28  0.28
   0.36 -0.43 -0.13  0.2  -0.75 -0.24  0.62 -0.99  0.12 -0.24  0.14  0.77
   0.98]
 [ 0.73  2.16 -1.94  2.07  0.35  0.45  1.12 -1.9   0.23 -0.4  -0.85  0.24
  -0.09  0.03 -0.29 -0.87  0.46  0.34 -0.07 -1.08  0.61 -0.33  0.33  0.3
   0.36 -0.48 -0.14  0.21 -0.73 -0.22  0.66 -0.94  0.14 -0.19  0.16  0.75
   1.  ]]
4160
[[ 0.69  2.17 -2.1   2.1   0.37  0.44  1.17 -1.86  0.18 -0.36

4320
[[ 0.7   2.18 -2.23  2.09  0.32  0.44  1.08 -1.86  0.21 -0.4  -0.92  0.32
  -0.11  0.05 -0.33 -0.85  0.4   0.33 -0.06 -1.07  0.65 -0.34  0.31  0.3
   0.37 -0.26 -0.14  0.2  -0.75 -0.26  0.62 -1.    0.11 -0.23  0.13  0.78
   0.99]
 [ 0.73  2.16 -1.94  2.07  0.35  0.45  1.12 -1.9   0.23 -0.4  -0.85  0.24
  -0.09  0.03 -0.29 -0.87  0.46  0.34 -0.07 -1.08  0.61 -0.33  0.33  0.3
   0.36 -0.48 -0.14  0.21 -0.73 -0.22  0.66 -0.94  0.14 -0.19  0.16  0.75
   1.  ]]
4330
[[ 0.71  2.15 -2.23  2.09  0.38  0.42  1.07 -1.83  0.17 -0.4  -0.88  0.34
  -0.14  0.01 -0.31 -0.87  0.39  0.34 -0.07 -1.05  0.68 -0.31  0.31  0.31
   0.38 -0.32 -0.09  0.23 -0.73 -0.24  0.61 -0.96  0.12 -0.19  0.17  0.75
   0.99]
 [ 0.73  2.16 -1.94  2.07  0.35  0.45  1.12 -1.9   0.23 -0.4  -0.85  0.24
  -0.09  0.03 -0.29 -0.87  0.46  0.34 -0.07 -1.08  0.61 -0.33  0.33  0.3
   0.36 -0.48 -0.14  0.21 -0.73 -0.22  0.66 -0.94  0.14 -0.19  0.16  0.75
   1.  ]]
4340
[[ 0.63  2.12 -2.28  2.06  0.34  0.42  1.1  -1.9   0.16 -0.37 

4500
[[ 0.7   2.22 -2.41  2.11  0.42  0.45  1.14 -1.83  0.25 -0.39 -0.93  0.37
  -0.11 -0.02 -0.33 -0.89  0.4   0.32 -0.07 -1.1   0.73 -0.37  0.27  0.3
   0.34 -0.31 -0.16  0.18 -0.77 -0.26  0.6  -1.01  0.1  -0.24  0.12  0.77
   0.98]
 [ 0.73  2.16 -1.94  2.07  0.35  0.45  1.12 -1.9   0.23 -0.4  -0.85  0.24
  -0.09  0.03 -0.29 -0.87  0.46  0.34 -0.07 -1.08  0.61 -0.33  0.33  0.3
   0.36 -0.48 -0.14  0.21 -0.73 -0.22  0.66 -0.94  0.14 -0.19  0.16  0.75
   1.  ]]
4510
[[ 0.7   2.22 -2.38  2.12  0.37  0.46  1.12 -1.81  0.2  -0.38 -0.9   0.39
  -0.1   0.02 -0.32 -0.87  0.39  0.33 -0.08 -1.12  0.7  -0.37  0.29  0.29
   0.37 -0.35 -0.11  0.2  -0.75 -0.25  0.61 -0.98  0.11 -0.23  0.14  0.79
   0.98]
 [ 0.73  2.16 -1.94  2.07  0.35  0.45  1.12 -1.9   0.23 -0.4  -0.85  0.24
  -0.09  0.03 -0.29 -0.87  0.46  0.34 -0.07 -1.08  0.61 -0.33  0.33  0.3
   0.36 -0.48 -0.14  0.21 -0.73 -0.22  0.66 -0.94  0.14 -0.19  0.16  0.75
   1.  ]]
4520
[[ 0.72  2.21 -2.35  2.1   0.4   0.43  1.13 -1.8   0.18 -0.39 

4680
[[ 0.72  2.19 -2.23  2.09  0.4   0.45  1.15 -1.89  0.2  -0.41 -0.89  0.34
  -0.09  0.01 -0.31 -0.88  0.44  0.33 -0.08 -1.08  0.7  -0.36  0.28  0.29
   0.34 -0.42 -0.14  0.22 -0.73 -0.23  0.63 -0.96  0.14 -0.22  0.15  0.76
   0.95]
 [ 0.73  2.16 -1.94  2.07  0.35  0.45  1.12 -1.9   0.23 -0.4  -0.85  0.24
  -0.09  0.03 -0.29 -0.87  0.46  0.34 -0.07 -1.08  0.61 -0.33  0.33  0.3
   0.36 -0.48 -0.14  0.21 -0.73 -0.22  0.66 -0.94  0.14 -0.19  0.16  0.75
   1.  ]]
4690
[[ 0.67  2.16 -2.23  2.09  0.39  0.46  1.11 -1.95  0.2  -0.4  -0.88  0.33
  -0.11  0.   -0.33 -0.9   0.47  0.34 -0.04 -1.05  0.67 -0.35  0.29  0.29
   0.39 -0.41 -0.13  0.23 -0.73 -0.21  0.63 -0.96  0.14 -0.21  0.16  0.75
   0.99]
 [ 0.73  2.16 -1.94  2.07  0.35  0.45  1.12 -1.9   0.23 -0.4  -0.85  0.24
  -0.09  0.03 -0.29 -0.87  0.46  0.34 -0.07 -1.08  0.61 -0.33  0.33  0.3
   0.36 -0.48 -0.14  0.21 -0.73 -0.22  0.66 -0.94  0.14 -0.19  0.16  0.75
   1.  ]]
4700
[[ 0.7   2.14 -2.17  2.09  0.38  0.42  1.1  -2.03  0.19 -0.42

4860
[[ 0.69  2.17 -2.07  2.07  0.38  0.4   1.08 -1.84  0.21 -0.4  -0.87  0.28
  -0.1   0.01 -0.31 -0.85  0.47  0.33 -0.04 -1.06  0.64 -0.32  0.29  0.34
   0.38 -0.52 -0.11  0.17 -0.77 -0.27  0.58 -0.99  0.09 -0.25  0.13  0.78
   0.96]
 [ 0.73  2.16 -1.94  2.07  0.35  0.45  1.12 -1.9   0.23 -0.4  -0.85  0.24
  -0.09  0.03 -0.29 -0.87  0.46  0.34 -0.07 -1.08  0.61 -0.33  0.33  0.3
   0.36 -0.48 -0.14  0.21 -0.73 -0.22  0.66 -0.94  0.14 -0.19  0.16  0.75
   1.  ]]
4870
[[ 0.68  2.15 -2.02  2.05  0.39  0.44  1.16 -1.85  0.21 -0.39 -0.87  0.26
  -0.09  0.   -0.33 -0.91  0.49  0.32 -0.05 -1.07  0.65 -0.33  0.3   0.32
   0.36 -0.51 -0.12  0.22 -0.71 -0.22  0.65 -0.95  0.16 -0.21  0.18  0.76
   0.96]
 [ 0.73  2.16 -1.94  2.07  0.35  0.45  1.12 -1.9   0.23 -0.4  -0.85  0.24
  -0.09  0.03 -0.29 -0.87  0.46  0.34 -0.07 -1.08  0.61 -0.33  0.33  0.3
   0.36 -0.48 -0.14  0.21 -0.73 -0.22  0.66 -0.94  0.14 -0.19  0.16  0.75
   1.  ]]
4880
[[ 0.7   2.15 -1.98  2.08  0.39  0.48  1.17 -1.87  0.21 -0.41

In [43]:
##推定されたパラメータの表示
#パラメータの事後平均
RS = np.arange(burnin, int(R/keep))
beta = np.mean(BETA[RS, ], axis=0)
Cov = np.mean(COV[:, :, RS], axis=2)

In [44]:
#真の回帰パラメータとの比較
pd.DataFrame({"推定パラメータ": beta, "真のパラメータ": betat})

,推定パラメータ,真のパラメータ
0,0.705721,0.730047
1,2.160315,2.164836
2,-2.155361,-1.940648
3,2.085627,2.071042
4,0.370791,0.351363
5,0.442761,0.453153
6,1.112803,1.124399
7,-1.927143,-1.896923
8,0.202306,0.234631
9,-0.396872,-0.404447


In [45]:
#真の相関行列との比較
pd.DataFrame(np.vstack((Cov, Covt)))

,0,1,2,3,4,5,6,7,8
0,1.000000,0.202649,0.336329,-0.240613,-0.031812,-0.184046,-0.041711,-0.176501,-0.108535
1,0.202649,1.000000,0.314315,0.215609,-0.283894,0.176827,-0.118302,-0.154023,0.312746
2,0.336329,0.314315,1.000000,-0.126209,0.259376,0.153284,0.234218,0.031415,-0.052645
3,-0.240613,0.215609,-0.126209,1.000000,0.017499,0.148274,0.057991,0.021867,0.099420
4,-0.031812,-0.283894,0.259376,0.017499,1.000000,-0.120707,0.080006,0.491617,-0.288442
5,-0.184046,0.176827,0.153284,0.148274,-0.120707,1.000000,0.458187,-0.137841,0.139451
6,-0.041711,-0.118302,0.234218,0.057991,0.080006,0.458187,1.000000,-0.083205,-0.266859
7,-0.176501,-0.154023,0.031415,0.021867,0.491617,-0.137841,-0.083205,1.000000,0.067560
8,-0.108535,0.312746,-0.052645,0.099420,-0.288442,0.139451,-0.266859,0.067560,1.000000
9,1.000000,0.236991,0.355446,-0.220743,-0.033425,-0.149735,-0.038546,-0.168662,-0.072048
